In [314]:
import pandas as pd
import sklearn as skt
import re
import os
from datetime import datetime, timezone
import matplotlib.pyplot as plt

In [315]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'FakeReal.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('titulo,URL\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath

In [316]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=True, index_label='index')
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=True, index_label='index')

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

In [317]:
df = pd.read_csv('db/concat_FatoFake.csv')
df.head()

,index,titulo,resumo,URL
0,0,É #FAKE vídeo de Paolla Oliveira anunciando de...,é fake post que usa vídeo de paolla oliveira p...,https://g1.globo.com/fato-ou-fake/video/e-fake...
1,1,É #FAKE que vídeo mostre criminosos deixando d...,polícia civil da bahia onde foi feita a gravaç...,https://g1.globo.com/fato-ou-fake/noticia/2025...
2,2,É #FATO: vídeo viral mostra treinamento milita...,publicada em fevereiro de na conta oficial de ...,https://g1.globo.com/fato-ou-fake/noticia/2025...
3,3,É #FAKE que mamografia aumenta o risco de cânc...,integrante da sociedade brasileira de mastolog...,https://g1.globo.com/fato-ou-fake/noticia/2025...
4,4,É #FAKE vídeo em que famosos usam camiseta com...,autor do registro é criador de conteúdos que u...,https://g1.globo.com/fato-ou-fake/noticia/2025...


In [318]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4565 entries, 0 to 4564
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   4565 non-null   int64 
 1   titulo  4565 non-null   object
 2   resumo  4563 non-null   object
 3   URL     4565 non-null   object
dtypes: int64(1), object(3)
memory usage: 142.8+ KB


In [319]:
# df = df.dropna(subset=['resumo'])
df['resumo'] = df['resumo'].fillna('não há dados sobre')


In [320]:
# Definir uma função para verificar se as frases específicas estão no 'resumo'
def remove_noticias_irrelevantes_pro_contexto(titulo):
    return ("estreia quadro no \'Mais Você\'" not in titulo) and ('apresenta o quadro \"Fato ou Fake\"' not in titulo)


In [321]:
# Aplicar a função para filtrar as linhas
df = df[df['titulo'].apply(remove_noticias_irrelevantes_pro_contexto)]

In [322]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4563 entries, 0 to 4564
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   4563 non-null   int64 
 1   titulo  4563 non-null   object
 2   resumo  4563 non-null   object
 3   URL     4563 non-null   object
dtypes: int64(1), object(3)
memory usage: 178.2+ KB


In [323]:
keys = {
    'fake': ['#FAKE', '#NÃO É BEM ASSIM']
    }

In [324]:
# Função para classificar
def classificar(row):
    # Verificar nas colunas 'titulo' e 'resumo'
    texto_completo = row['titulo'] + " " + row['resumo']  # Junta as duas colunas em uma string
    # Verifica se alguma palavra-chave de 'fake' aparece
    if any(palavra in texto_completo for palavra in keys['fake']):
        return 'fake'
    return 'fato'  # Se não encontrar 'fake', classifica como 'fato'


In [325]:
df['check'] = df.apply(classificar, axis=1)
df.head()


,index,titulo,resumo,URL,check
0,0,É #FAKE vídeo de Paolla Oliveira anunciando de...,é fake post que usa vídeo de paolla oliveira p...,https://g1.globo.com/fato-ou-fake/video/e-fake...,fake
1,1,É #FAKE que vídeo mostre criminosos deixando d...,polícia civil da bahia onde foi feita a gravaç...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake
2,2,É #FATO: vídeo viral mostra treinamento milita...,publicada em fevereiro de na conta oficial de ...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fato
3,3,É #FAKE que mamografia aumenta o risco de cânc...,integrante da sociedade brasileira de mastolog...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake
4,4,É #FAKE vídeo em que famosos usam camiseta com...,autor do registro é criador de conteúdos que u...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake


In [326]:
df['check'].value_counts(normalize=True)*100

check
fake    60.815253
fato    39.184747
Name: proportion, dtype: float64

In [327]:
print(df['titulo'][df['check'] == 'fato'])

2       É #FATO: vídeo viral mostra treinamento milita...
40      Trump deu declarações falsas e enganosas duran...
47      Dá para contrair HPV em aparelho da academia? ...
153     Atores são pagos para sair às ruas e espalhar ...
179     Juiz suspende deepfake em perfil do TikTok em ...
                              ...                        
4560    Empresário é preso em Vitória por ligação com ...
4561    Lula estuda novo xadrez ministerial com ajuste...
4562    Compras internacionais recuam em 2024, mas arr...
4563    Tom Welling, de 'Smallville', é preso nos Esta...
4564    VÍDEO: Polícia prende em flagrante dois ladrõe...
Name: titulo, Length: 1788, dtype: object


In [328]:
print(df['resumo'][df['check'] == 'fato'])


2       publicada em fevereiro de na conta oficial de ...
40      após ser empossado nesta segunda ele repetiu v...
47      jovem diz que contraiu hpv na mão por equipame...
153     grupo era contratado por políticos em períodos...
179     equipe do fato ou fake apurou e concluiu com d...
                              ...                        
4560    "Operação Conexão Perdida" prendeu duas pessoa...
4561    Expectativa de aliados é de que presidente tro...
4562    Em agosto, governo aumentou para 20% o imposto...
4563    O ator de 47 anos foi preso no domingo (26) e ...
4564    Ação aconteceu na Avenida Lins Vasconcelos na ...
Name: resumo, Length: 1788, dtype: object


In [329]:
saveData(df)

Arquivo base criado ou já existente em: e:\CODES_01\PYTHON\NPL\fatoXfake\FakeReal\db\FakeReal.csv


Dados salvos no arquivo 'e:\CODES_01\PYTHON\NPL\fatoXfake\FakeReal\db\FakeReal.csv' com sucesso!
